In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
from matplotlib import pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score

In [2]:
d = open('./final_combined_dataset.csv','r')
data_lst = [line.strip('\n').replace('"','').split(',') for line in d.readlines()]
data = pd.DataFrame(data_lst, columns = data_lst[0])
pd.set_option('display.max_columns', None)
data = data.drop(index = 0)
data = data.reset_index()
data = data.apply(pd.to_numeric,errors = 'ignore')
data.set_index(["index"], inplace=True)
data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,nursery,internet,guardian.x,traveltime.x,studytime.x,failures.x,schoolsup.x,famsup.x,paid.x,activities.x,higher.x,romantic.x,famrel.x,freetime.x,goout.x,Dalc.x,Walc.x,health.x,absences.x,G1.x,G2.x,G3.x,guardian.y,traveltime.y,studytime.y,failures.y,schoolsup.y,famsup.y,paid.y,activities.y,higher.y,romantic.y,famrel.y,freetime.y,goout.y,Dalc.y,Walc.y,health.y,absences.y,G1.y,G2.y,G3.y
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,15,0,0,1,1,1,0,2,1,1,1,1,2,4,1,1,1,1,1,1,0,3,1,2,1,1,1,2,7,10,10,1,2,4,0,1,1,1,1,1,0,3,1,2,1,1,1,4,13,13,13
2,0,0,15,0,0,1,1,1,2,2,3,0,1,1,1,2,2,1,1,0,0,1,1,3,3,4,2,4,5,2,8,6,5,1,1,2,0,1,1,0,0,1,1,3,3,4,2,4,5,2,13,11,11
3,0,0,15,0,0,1,2,2,0,2,3,1,0,1,1,1,0,1,1,1,1,1,0,4,3,1,1,1,2,8,14,13,13,1,1,1,0,1,1,0,1,1,0,4,3,1,1,1,2,8,14,13,12
4,0,0,15,0,0,1,2,4,3,1,0,1,1,1,1,3,0,1,1,1,1,1,0,4,3,2,1,1,5,2,10,9,8,1,1,3,0,1,1,0,1,1,0,4,3,2,1,1,5,2,10,11,10
5,0,0,15,0,0,1,3,3,3,3,3,1,1,2,2,3,2,0,1,1,1,1,1,4,2,1,2,3,3,8,10,10,10,2,2,3,0,0,1,1,1,1,1,4,2,1,2,3,3,2,13,13,13


In [3]:
G1x_binary = []
for g1 in data['G1.x']:
    if g1 < 10:
        G1x_binary.append(1)
    else:
        G1x_binary.append(0)
data['G1x_binary'] = G1x_binary
G2x_binary = []
for g2 in data['G2.x']:
    if g1 < 10:
        G2x_binary.append(1)
    else:
        G2x_binary.append(0)
data['G2x_binary'] = G2x_binary
G3x_binary = []
for g3 in data['G3.x']:
    if g3 < 10:
        G3x_binary.append(1)
    else:
        G3x_binary.append(0)
data['G3x_binary'] = G3x_binary

In [4]:
G1y_binary = []
for g1 in data['G1.y']:
    if g1 < 10:
        G1y_binary.append(1)
    else:
        G1y_binary.append(0)
data['G1y_binary'] = G1y_binary
G2y_binary = []
for g2 in data['G2.y']:
    if g1 < 10:
        G2y_binary.append(1)
    else:
        G2y_binary.append(0)
data['G2y_binary'] = G2y_binary
G3y_binary = []
for g3 in data['G3.y']:
    if g3 < 10:
        G3y_binary.append(1)
    else:
        G3y_binary.append(0)
data['G3y_binary'] = G3y_binary
data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,nursery,internet,guardian.x,traveltime.x,studytime.x,failures.x,schoolsup.x,famsup.x,paid.x,activities.x,higher.x,romantic.x,famrel.x,freetime.x,goout.x,Dalc.x,Walc.x,health.x,absences.x,G1.x,G2.x,G3.x,guardian.y,traveltime.y,studytime.y,failures.y,schoolsup.y,famsup.y,paid.y,activities.y,higher.y,romantic.y,famrel.y,freetime.y,goout.y,Dalc.y,Walc.y,health.y,absences.y,G1.y,G2.y,G3.y,G1x_binary,G2x_binary,G3x_binary,G1y_binary,G2y_binary,G3y_binary
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,15,0,0,1,1,1,0,2,1,1,1,1,2,4,1,1,1,1,1,1,0,3,1,2,1,1,1,2,7,10,10,1,2,4,0,1,1,1,1,1,0,3,1,2,1,1,1,4,13,13,13,1,1,0,0,0,0
2,0,0,15,0,0,1,1,1,2,2,3,0,1,1,1,2,2,1,1,0,0,1,1,3,3,4,2,4,5,2,8,6,5,1,1,2,0,1,1,0,0,1,1,3,3,4,2,4,5,2,13,11,11,1,1,1,0,0,0
3,0,0,15,0,0,1,2,2,0,2,3,1,0,1,1,1,0,1,1,1,1,1,0,4,3,1,1,1,2,8,14,13,13,1,1,1,0,1,1,0,1,1,0,4,3,1,1,1,2,8,14,13,12,0,1,0,0,0,0
4,0,0,15,0,0,1,2,4,3,1,0,1,1,1,1,3,0,1,1,1,1,1,0,4,3,2,1,1,5,2,10,9,8,1,1,3,0,1,1,0,1,1,0,4,3,2,1,1,5,2,10,11,10,0,1,1,0,0,0
5,0,0,15,0,0,1,3,3,3,3,3,1,1,2,2,3,2,0,1,1,1,1,1,4,2,1,2,3,3,8,10,10,10,2,2,3,0,0,1,1,1,1,1,4,2,1,2,3,3,2,13,13,13,0,1,0,0,0,0


In [5]:
label_encoder = preprocessing.LabelEncoder()
data_to_numeric_x = ['school','sex','address','famsize','Pstatus','Mjob','Fjob','reason','guardian.x','nursery','internet','famsup.x']
demographic_feature_x = ['school','sex','age','address','famsize','Pstatus','Medu','Fedu','Mjob','Fjob','reason','guardian.x','nursery','internet','famsup.x']
data_demo_x = data[demographic_feature_x]
for i in data_to_numeric_x:
    data_demo_x[i]= label_encoder.fit_transform(data_demo_x[i])
data_demo_x.head()

c:\users\19501\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian.x,nursery,internet,famsup.x
index,,,,,,,,,,,,,,,
1,0,0,15,0,0,1,1,1,0,2,1,1,1,1,1
2,0,0,15,0,0,1,1,1,2,2,3,1,0,1,1
3,0,0,15,0,0,1,2,2,0,2,3,1,1,0,1
4,0,0,15,0,0,1,2,4,3,1,0,1,1,1,1
5,0,0,15,0,0,1,3,3,3,3,3,2,1,1,1


In [6]:
data['G1x_binary'] = label_encoder.fit_transform(data['G1x_binary'])
data['G2x_binary'] = label_encoder.fit_transform(data['G2x_binary'])
data['G3x_binary'] = label_encoder.fit_transform(data['G3x_binary'])
data['G1y_binary'] = label_encoder.fit_transform(data['G1y_binary'])
data['G2y_binary'] = label_encoder.fit_transform(data['G2y_binary'])
data['G3y_binary'] = label_encoder.fit_transform(data['G3y_binary'])

In [7]:
train_x = data_demo_x
trainG1_y = data.values[:,-6]
trainG3_y = data.values[:,-4]
trainG1_y=trainG1_y.astype('int')
trainG3_y=trainG3_y.astype('int')
x_trainG1,x_testG1,y_trainG1,y_testG1 = train_test_split(train_x,trainG1_y,test_size=0.25,random_state=0)
x_trainG3,x_testG3,y_trainG3,y_testG3 = train_test_split(train_x,trainG3_y,test_size=0.25,random_state=0)

###### LR-G1

In [8]:
from sklearn.linear_model import LogisticRegressionCV
LR = LogisticRegressionCV(class_weight="balanced", cv=10,verbose=1,n_jobs=-1,scoring='roc_auc',solver='newton-cg',penalty='l2')
LRmodel1 = LR.fit(x_trainG1,y_trainG1)
predLR1 = LRmodel.predict(x_testG1)
accuracyLR1 = accuracy_score(y_testG1,predLR1)
print("Accuracy: ", accuracyLR1)
print(classification_report(y_testG1, predLR1))
print("Kappa Score: " +str(cohen_kappa_score(predLR1,y_testG1)))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    1.4s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    1.5s finished


NameError: name 'LRmodel' is not defined

###### LR-G3

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
LR = LogisticRegressionCV(class_weight="balanced", cv=10,verbose=1,n_jobs=-1,scoring='roc_auc',solver='newton-cg',penalty='l2')
LRmodel3 = LR.fit(x_trainG3,y_trainG3)
predLR3 = LRmodel.predict(x_testG3)
accuracyLR3 = accuracy_score(y_testG3,predLR3)
print("Accuracy: ", accuracyLR3)
print(classification_report(y_testG3, predLR3))
print("Kappa Score: " +str(cohen_kappa_score(predLR3,y_testG3)))

In [ ]:
probaLRG1 = LRmodel1.predict_proba(x_testG1)
probaLRG3 = LRmodel3.predict_proba(x_testG3)
proba_LRG1 = []
for i in range(len(probaLRG1)):
    proba_LRG1.append(round(probaLRG1[i][1],4))
proba_LRG3 = []
for i in range(len(probaLRG3)):
    proba_LRG3.append(round(probaLRG3[i][1],4))
X_test_data = x_testG1.copy()
X_test_data["proba_LRG1"] = proba_LRG1
X_test_data["proba_LRG3"] = proba_LRG3

In [ ]:
from pandas import Series,DataFrame
df = {"demographic_feature":demographic_feature_x}
feature_importances = DataFrame(df)

importanceLRG1 = LRmodel1.coef_
importanceLRG3 = LRmodel3.coef_

importance_LRG1 = []
for i in range(len(importanceLRG1[0])):
    importance_LRG1.append(round(importanceLRG1[0][i],4))
feature_importances['importance_LRG1'] = importance_LRG1
importance_LRG3 = []
for i in range(len(importanceLRG3[0])):
    importance_LRG3.append(round(importanceLRG3[0][i],4))
feature_importances['importance_LRG3'] = importance_LRG3

###### GNB

In [ ]:
from sklearn.naive_bayes import GaussianNB
GNB = GaussianNB()
GNB1 = GNB.fit(x_trainG1,y_trainG1)
predGNB1 = GNB1.predict(x_testG1)
accuracyGNB1 = accuracy_score(y_testG1,predGNB1)
print("Accuracy: ",accuracyGNB1)
print(classification_report(y_testG1, predGNB1))
print("Kappa Score: " +str(cohen_kappa_score(predGNB1,y_testG1)))

In [ ]:
GNB3 = GNB.fit(x_trainG1,y_trainG3)
predGNB3 = GNB3.predict(x_testG3)
accuracyGNB3 = accuracy_score(y_testG3,predGNB3)
print("Accuracy: ",accuracyGNB3)
print(classification_report(y_testG3, predGNB3))
print("Kappa Score: " +str(cohen_kappa_score(predGNB3,y_testG3)))

In [ ]:
probaGNBG1 = GNB1.predict_proba(x_testG1)
probaGNBG3 = GNB3.predict_proba(x_testG3)
proba_GNBG1 = []
for i in range(len(probaGNBG1)):
    proba_GNBG1.append(round(probaGNBG1[i][1],4))
proba_GNBG3 = []
for i in range(len(probaGNBG3)):
    proba_GNBG3.append(round(probaGNBG3[i][1],4))
X_test_data["proba_GNBG1"] = proba_GNBG1
X_test_data["proba_GNBG3"] = proba_GNBG3

In [ ]:
X_test_data.shape

###### LSVM

In [ ]:
from sklearn import svm
from sklearn.svm import SVC
linear_svc = svm.SVC(kernel='linear', probability = True)
LSVM1 = linear_svc.fit(x_trainG1,y_trainG1)
predSVC1 = linear_svc.predict(x_testG1)
accuracySVC1 = accuracy_score(y_testG1,predSVC1)
print("Accuracy: ",accuracySVC1)
print(classification_report(y_testG1, predSVC1))
print("Kappa Score: " +str(cohen_kappa_score(predSVC1,y_testG1)))

In [ ]:
LSVM3 = linear_svc.fit(x_trainG3,y_trainG3)
predSVC3 = linear_svc.predict(x_testG3)
accuracySVC3 = accuracy_score(y_testG3,predSVC3)
print("Accuracy: ",accuracySVC3)
print(classification_report(y_testG3, predSVC3))
print("Kappa Score: " +str(cohen_kappa_score(predSVC3,y_testG3)))

In [ ]:
probaLSVMG1 = LSVM1.predict_proba(x_testG1)
probaLSVMG3 = LSVM3.predict_proba(x_testG3)
proba_LSVMG1 = []
for i in range(len(probaLSVMG1)):
    proba_LSVMG1.append(round(probaLSVMG1[i][1],4))
proba_LSVMG3 = []
for i in range(len(probaLSVMG3)):
    proba_LSVMG3.append(round(probaLSVMG3[i][1],4))
X_test_data["proba_LSVMG1"] = proba_LSVMG1
X_test_data["proba_LSVMG3"] = proba_LSVMG3

###### DCT

In [ ]:
from sklearn.tree import DecisionTreeClassifier
DTC = DecisionTreeClassifier(max_depth = 5)
DTC1 = DTC.fit(x_trainG1,y_trainG1)
predDTC1 = DTC.predict(x_testG1)
accuracyDTC1 = accuracy_score(y_testG1,predDTC1)
print("Accuracy: ",accuracyDTC1)
print(classification_report(y_testG1, predDTC1))
print("Kappa Score: " +str(cohen_kappa_score(predDTC1,y_testG1)))

In [ ]:
DTC3 = DTC.fit(x_trainG3,y_trainG3)
predDTC3 = DTC.predict(x_testG3)
accuracyDTC3 = accuracy_score(y_testG3,predDTC3)
print("Accuracy: ",accuracyDTC3)
print(classification_report(y_testG3, predDTC3))
print("Kappa Score: " +str(cohen_kappa_score(predDTC3,y_testG3)))

In [ ]:
probaDTCG1 = DTC1.predict_proba(x_testG1)
probaDTCG3 = DTC3.predict_proba(x_testG3)
proba_DTCG1 = []
for i in range(len(probaDTCG1)):
    proba_DTCG1.append(round(probaDTCG1[i][1],4))
proba_DTCG3 = []
for i in range(len(probaDTCG3)):
    proba_DTCG3.append(round(probaDTCG3[i][1],4))
X_test_data["proba_DTCG1"] = proba_DTCG1
X_test_data["proba_DTCG3"] = proba_DTCG3

In [ ]:
importanceDTCG1 = DTC1.feature_importances_
importanceDTCG3 = DTC3.feature_importances_

importance_DTCG1 = []
for i in range(len(importanceDTCG1)):
    importance_DTCG1.append(round(importanceDTCG1[i],4))
feature_importances['importance_DTCG1'] = importance_DTCG1
importance_DTCG3 = []
for i in range(len(importanceDTCG3)):
    importance_DTCG3.append(round(importanceDTCG3[i],4))
feature_importances['importance_DTCG3'] = importance_DTCG3

###### RF

In [ ]:
RF = RandomForestClassifier()
RF1 = RF.fit(x_trainG1, y_trainG1)
predRF1 = RF.predict(x_testG1)
accuracyRF1 = accuracy_score(y_testG1,predRF1)
print("Accuracy: ",accuracyRF1)
print(classification_report(y_testG1, predRF1))
print("Kappa Score: " +str(cohen_kappa_score(predRF1,y_testG1)))

In [ ]:
RF3 = RF.fit(x_trainG3, y_trainG3)
predRF3 = RF.predict(x_testG3)
accuracyRF3 = accuracy_score(y_testG3,predRF3)
print("Accuracy: ",accuracyRF3)
print(classification_report(y_testG3, predRF3))
print("Kappa Score: " +str(cohen_kappa_score(predRF3,y_testG3)))

In [ ]:
probaRFG1 = RF1.predict_proba(x_testG1)
probaRFG3 = RF3.predict_proba(x_testG3)
proba_RFG1 = []
for i in range(len(probaRFG1)):
    proba_RFG1.append(round(probaRFG1[i][1],4))
proba_RFG3 = []
for i in range(len(probaRFG3)):
    proba_RFG3.append(round(probaRFG3[i][1],4))
X_test_data["proba_RFG1"] = proba_RFG1
X_test_data["proba_RFG3"] = proba_RFG3

In [ ]:
X_test_data.shape

In [ ]:
importanceRFG1 = RF1.feature_importances_
importanceRFG3 = RF3.feature_importances_

importance_RFG1 = []
for i in range(len(importanceRFG1)):
    importance_RFG1.append(round(importanceRFG1[i],4))
feature_importances['importance_RFG1'] = importance_DTCG1
importance_RFG3 = []
for i in range(len(importanceRFG3)):
    importance_RFG3.append(round(importanceRFG3[i],4))
feature_importances['importance_RFG3'] = importance_RFG3

In [ ]:
feature_importances